# AlphaJet - MVP3 
## Part #1 - Grab latest BTC price and create MAs, Bollinger bands and RSI features
#### Sources : 

##### ML for Algo Trading - Stefan Jansen book
##### Medium article "Retrieving Full Historical Data for Every Cryptocurrency on Binance & BitMex Using the Python API" from Peter Nistrup
https://betterprogramming.pub/easiest-way-to-use-the-bitmex-api-with-python-fbf66dc38633

In [1]:
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import talib
import quandl
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import yfinance as yf
from datetime import date,timedelta, datetime
import math
import os.path
import time
from bitmex import bitmex
from binance.client import Client
from dateutil import parser
from tqdm import tqdm_notebook #(Optional, used for progress-bars)
#import pandas_profiling #Optional - for fancy data analysis reports!

#### Defining API keys for Binance and Bitmex

In [2]:
### API
bitmex_api_key = 'REDACTED'    #Enter your own API-key here
bitmex_api_secret = '[REDACTED]' #Enter your own API-secret here
binance_api_key = 'VaBzOEyR5SpRLCkWg2CBuBnlsAP6I0nEuVOFuO4AHO1IrDg7H3WNo2IizWBfCr03'    #Enter your own API-key here
binance_api_secret = 'lgDl4hN89eJ8P7BhhHKQqX9DBTrA3peK4fDgJj6L8wsdyonCYI1XJTOoEs60IyRO' #Enter your own API-secret here

#### Constants

In [3]:
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
batch_size = 750
#bitmex_client = bitmex(test=False, api_key=bitmex_api_key, api_secret=bitmex_api_secret)
binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)

### FUNCTIONS

In [4]:
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('1 Jan 2017', '%d %b %Y')
    elif source == "bitmex": old = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=False).result()[0][0]['timestamp']
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    if source == "bitmex": new = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new

def get_all_binance(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    if oldest_point == datetime.strptime('1 Jan 2017', '%d %b %Y'): print('Downloading all available %s data for %s. Be patient..!' % (kline_size, symbol))
    else: print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
    klines = binance_client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = pd.concat([data_df, temp_df])
        #data_df = data_df.append(temp_df)
    else: data_df = data
    data_df.set_index('timestamp', inplace=True)
    if save: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

def get_all_bitmex(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "bitmex")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    rounds = math.ceil(available_data / batch_size)
    if rounds > 0:
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data in %d rounds.' % (delta_min, symbol, available_data, kline_size, rounds))
        for round_num in tqdm_notebook(range(rounds)):
            time.sleep(1)
            new_time = (oldest_point + timedelta(minutes = round_num * batch_size * binsizes[kline_size]))
            data = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=batch_size, startTime = new_time).result()[0]
            temp_df = pd.DataFrame(data)
            data_df = pd.concat([data_df, temp_df])
            #data_df = data_df.append(temp_df)
    data_df.set_index('timestamp', inplace=True)
    if save and rounds > 0: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

# 1) Download Data


##### Getting the Data from Binance

In [5]:
data = get_all_binance('ETHUSDT','1h',save=True)

All caught up..!


In [13]:
data = get_all_binance('ETHUSDT','1h',save=True)

/var/folders/x7/v571_3t12ddffkm6dh_29_tr0000gn/T/ipykernel_32597/3820235638.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)


All caught up..!


In [14]:
# you can dowload fear and grid data from https://api.alternative.me/fng/?limit=30
import requests
import pandas as pd

# API endpoint
url = "https://api.alternative.me/fng/?limit=0"

# Send request and get the response
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON data
    data = response.json()["data"]
    
    # Create a DataFrame
    df = pd.DataFrame(data)
    
    # Convert timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    
    # Set datetime as index
    df.set_index('timestamp', inplace=True)

    # Save the DataFrame to a CSV file
    df.to_csv("crypto_fear_and_greed_index.csv")
    
    print("Data downloaded and saved as crypto_fear_and_greed_index.csv")
else:
    print(f"Error: {response.status_code}")


Data downloaded and saved as crypto_fear_and_greed_index.csv


In [ ]:
data.head(3)
data.info()

In [ ]:
data.tail(3)

##### using the last 30 days and cleaning up the original BTC file

In [ ]:
#start_date = pd.to_datetime('5/1/2020 00:00', utc= True)
#end_date = pd.to_datetime('8/1/2021 00:00', utc= True)
#end_date = pd.to_datetime(datetime.now(), utc= True)
#start_date

In [ ]:
# converting index to proper datetime object
data.index = pd.to_datetime(data.index)
data.index = data.index.tz_localize('utc')

In [ ]:
#remove any duplicate
data = data.groupby(data.index).first()

In [ ]:
#price = data[(data.index >= start_date) & (data.index <= end_date)]
price = data.tail(7600)

price=price[['open','high','low','close','volume']]


# Converts the header name to all uppercase
price.columns = [x.capitalize() for x in price.columns] 

# Add AdjClose column
price['AdjClose']=price['Close']

# Changing columns to float
price = price.astype(float)

# Renaming index
price.index.rename('Date', inplace=True)

price.head()

In [ ]:
price.dtypes

#### Adding Moving Averages

In [ ]:
feature_names = ['5d_close_pct']  # a list of the feature names for later

# Create moving averages and rsi for timeperiods of 5, 7, 14, 30, 50,100,200,350,700
for n in [5,7,14,20,30,50,100,200,350,600,700]:

    # Create the simple moving average indicator and divide by Adj_Close
    price['ma' + str(n)] = talib.SMA(price['AdjClose'].values, timeperiod=n) 
    price['tsma'+ str(n)] = talib.TSF(price['AdjClose'].values, timeperiod=n)
    price['tsma_norm'+ str(n)] = talib.TSF(price['AdjClose'].values, timeperiod=n)/price['AdjClose']
    
    # Create the exp moving average indicator and divide by Adj_Close
    price['ema' + str(n)] = talib.EMA(price['AdjClose'].values,timeperiod=n) 
    
    # Create the RSI indicator
    price['rsi' + str(n)] = talib.RSI(price['AdjClose'].values, timeperiod=n)
    
    # Add previous close price for the n previous days, and n-day percentage % price change
    price['previous'+ str(n)+'d_close'] = price['AdjClose'].shift(n) 
    price[str(n)+'d_close_pct'] = price['AdjClose'].pct_change(n)

    # Add rsi, moving average, tsma and previous days close prices and % change to the feature name list
    feature_names = feature_names + ['ma'+str(n),'tsma'+ str(n),'tsma_norm'+ str(n),'ema' + str(n), 'rsi' + str(n), 'previous'+str(n), str(n)+'d_close_pct']
    

print(feature_names)

price.tail()

In [ ]:
price.head()

#### Adding PPO and Bollinger Bands

In [ ]:
close = price['AdjClose'].values
up, mid, low = talib.BBANDS(close, timeperiod=14, nbdevup=2, nbdevdn=2, matype=0)

In [ ]:
# calculate Bollinger bands for time period 14 days, using TAlib library 

up, mid, low = talib.BBANDS(close, timeperiod=14, nbdevup=2, nbdevdn=2, matype=0)
price['BB_up'] = up
price['BB_mid'] = mid
price['BB_low'] = low
price['BBP14'] = (price['AdjClose'] - low) / (up - low)
price['BB_up_norm'] = up/price['AdjClose']
price['BB_mid_norm'] = mid/price['AdjClose']
price['BB_low_norm'] = low/price['AdjClose']
price['BBP14_norm'] = price['BBP14']/price['AdjClose']


# Add ppo indicators

# create PPO Percentage Price Oscillator - normalized version of MACD for 20-day and 50-day exp moving average
price['ppo20-50'] = talib.PPO(price['AdjClose'].values, fastperiod=20, slowperiod=50, matype=0) 
price['ppo50-100'] = talib.PPO(price['AdjClose'].values, fastperiod=50, slowperiod=100, matype=0)
price['ppo7-20'] = talib.PPO(price['AdjClose'].values, fastperiod=7, slowperiod=20, matype=0) 
price['ppo100-200'] = talib.PPO(price['AdjClose'].values, fastperiod=100, slowperiod=200, matype=0) 
price['ppo200-350'] = talib.PPO(price['AdjClose'].values, fastperiod=200, slowperiod=350, matype=0) 
price['ppo100-700'] = talib.PPO(price['AdjClose'].values, fastperiod=100, slowperiod=700, matype=0) 
price['ppo20-200'] = talib.PPO(price['AdjClose'].values, fastperiod=20, slowperiod=200, matype=0) 
price['ppo50-350'] = talib.PPO(price['AdjClose'].values, fastperiod=50, slowperiod=350, matype=0) 

In [ ]:
price.columns

In [ ]:
price.tail(70)

## 2) Exporting to .csv

In [ ]:
# Exporting last 3 weeks of data
Latest=price

# datetime object containing current date and time
now = datetime.now()

# YYYY-mm-dd_HM
date_string = now.strftime("%Y-%m-%d")
#Latest.to_csv("datasets_to_score/BTC_binance_"+str(date_string)+".csv", index=True, header=True)
Latest.to_csv("datasets_to_score/BTC_binance_"+str(date_string)+".csv", index=True, header=True)

print("'BTC_binance_"+str(date_string)+".csv' has been exported")



In [ ]:
Latest